In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import absl
from absl import app
from absl import flags
from absl import logging

import json
from pathlib import Path

from nasbench.lib import config as nasbench_config
from nasbench.lib import model_spec
from nasbench.lib import cifar
from nasbench.lib import model_builder
from nasbench import api as nasbench_api
from nasbench.scripts.run_evaluation import NumpyEncoder

import numpy as np
import tensorflow as tf

from utils import train

from train_student import FLAGS

In [3]:
%load_ext line_profiler

In [4]:
with open('../data/student_data/keys_20.json', "r") as f:
    student_hash_keys = json.load(f)

student_key = student_hash_keys[0]
student_key

'02e5a0247bbdcf2860b7e96f74961594'

In [5]:
nasbench = nasbench_api.NASBench('../data/nasbench_only108.tfrecord')

Loading dataset from file... This may take a few minutes...
Loaded dataset in 36 seconds


In [6]:
def get_spc_cfg(student_key=student_key):
    module = nasbench.fixed_statistics[student_key]
    spec = model_spec.ModelSpec(module['module_adjacency'], module['module_operations'])

    config = nasbench_config.build_config()
    config['train_data_files'] = [
        "../data/dataset/train_1_KD_11.tfrecords",
        "../data/dataset/train_2_KD_11.tfrecords",
        "../data/dataset/train_3_KD_11.tfrecords",
        "../data/dataset/train_4_KD_11.tfrecords"
    ]
    config['valid_data_file'] = '../data/dataset/validation.tfrecords'
    config['test_data_file'] = '../data/dataset/test.tfrecords'
    config['sample_data_file'] = '../data/dataset/sample.tfrecords'
    config['batch_size'] = 128
    config['train_epochs'] = 108
    config['use_tpu'] = False
    config['use_KD'] = True
    config['intermediate_evaluations'] = ['1.0']
    config['imitation_lmb'] = 0.7
    config['temperature'] = 20
    
    return spec, config
#     train(spec, config, './tmp')

In [7]:
from nasbench.lib.evaluate import _TrainAndEvaluator

spec, config = get_spc_cfg()
evaluator = _TrainAndEvaluator(spec, config, './tmp')

INFO:tensorflow:Using config: {'_model_dir': './tmp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 1073741824, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8db596d9b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=2, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=None, input_pa

In [ ]:
%lprun -f _TrainAndEvaluator.run evaluator.run()





----------->
train_steps:  3749 


Instructions for updating:
Use `tf.data.experimental.shuffle_and_repeat(...)`.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:vertex_channels: [128, 128, 128, 128, 128, 128, 128]
INFO:tensorflow:vertex_channels: [128, 128, 128, 128, 128, 128, 128]
INFO:tensorflow:vertex_channels: [128, 128, 128, 128, 128, 128, 128]
INFO:tensorflow:vertex_channels: [128, 256, 256, 256, 256, 256, 256]
INFO:tensorflow:vertex_channels: [256, 256, 256, 256, 256, 256, 256]
INFO:tensorflow:vertex_channels: [256, 256, 256, 256, 256, 256, 256]
INFO:tensorflow:vertex_channels: [256, 512, 512, 512, 512, 512, 512]
INFO:tensorflow:vertex_channels: [512, 512, 512, 512, 512, 512, 512]
INFO:tensorflow:vertex_channels: [512, 512, 512, 512, 512, 512, 512]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finaliz